# Einführung in Gatechecks

Um die einzelnen Übergabe wie im Stafetenlauf zu überprüfen können wir mit Hilfe von Gate Checks die Outputs validieren

In [120]:
import os
import ast
import re
from openai import AzureOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, field_validator, ValidationError

# --- 1. Configuration and OpenAI Client Initialization ---

load_dotenv()

# Azure OpenAI Configuration
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

if not api_key or not azure_endpoint or not azure_deployment:
    raise ValueError("Azure OpenAI Konfiguration fehlt. Bitte überprüfe deine .env Datei.")

# Initialize the Azure OpenAI Client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- 2. Function for the actual LLM call ---

def call_llm(system_prompt: str, user_prompt: str, temperature: float = 0.0) -> str:
    """Performs an actual API call to the Azure OpenAI model."""
    try:
        response = client.chat.completions.create(
            model=azure_deployment,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature
        )
        content = response.choices[0].message.content
        # Clean up Markdown code blocks if the LLM adds them
        if content.strip().startswith("```python"):
            content = content.strip().replace("```python", "").replace("```", "").strip()
        return content
    except Exception as e:
        print(f"Fehler beim API-Aufruf: {e}")
        return f"API_ERROR: {e}"

# Gate Check Modelle
Wir überprüfen die Keranforderungen des Plans und die Syntax des generierten Codes

In [125]:

# Pydantic Models for Gate Checks

# Workflow Plan checkt ob die Schritte des Plans die Kernanforderungen abdecken

class WorkflowPlan(BaseModel):
    schritte: list[str]

    @field_validator('schritte')
    def plan_muss_kernschritte_enthalten(cls, schritte_liste: list[str]) -> list[str]:
        gesamter_text = " ".join(schritte_liste).lower()
        
        # Wir definieren Synonyme für jeden erforderlichen Schritt
        erforderliche_schritte = {
            "lesen": ["lesen", "öffnen", "einlesen"],
            "berechnen": ["berechnen", "kalkulieren", "ermitteln"],
            "schreiben": ["schreiben", "speichern", "erstellen"]
        }
        
        for hauptschritt, synonyme in erforderliche_schritte.items():
            # Prüfen, ob mindestens EINES der Synonyme im Text vorkommt
            if not any(re.search(r'\b' + synonym + r'\b', gesamter_text) for synonym in synonyme):
                raise ValueError(f"Der Plan muss einen Schritt zum Thema '{hauptschritt}' enthalten.")
        return schritte_liste

def validate_outline(text_to_validate: str) -> WorkflowPlan | None:
    """
    Prüft und parst den Umriss-Text. 
    Gibt ein validiertes WorkflowPlan-Objekt bei Erfolg zurück oder None bei einem Fehler.
    """
    try:
        outline_list = [line.strip() for line in text_to_validate.strip().split('\n') if line.strip()]
        
        # Das validierte Objekt wird direkt zurückgegeben
        validated_plan = WorkflowPlan(schritte=outline_list)
        print("Gate Check 1 (Plan-Struktur): Erfolgreich!")
        return validated_plan

    except ValidationError as e:
        print(f"Gate Check 1 (Plan-Struktur): Gescheitert! Fehler: {e}")
        return None


# SyntaxCheck prüft ob der generierte Code syntaktisch korrekt ist

class SyntaxCheck(BaseModel):
    """Gate Check 2: Validiert die Syntax des Codes (mit AST)."""
    code: str

    @field_validator('code')
    def code_must_have_valid_syntax(cls, the_code: str):
        try:
            ast.parse(the_code)
        except SyntaxError as e:
            raise ValueError(f"Ungültige Python-Syntax: {e}")
        print("Gate Check 2 (Code-Syntax via AST): Erfolgreich!")
        return the_code


def validate_and_refine_code(client, generated_code: str) -> str | None:
    """
    Validiert einen übergebenen Code-String. Wenn er fehlschlägt, wird ein
    """
    # 1: Prüfe das Ergebnis des ersten LLM-Aufrufs.
    if generated_code.startswith("API_ERROR:"):
        print("Prozess gestoppt, da der ursprüngliche API-Aufruf fehlgeschlagen ist.")
        return None

    try:
        # Schritt 1: Versuche, den übergebenen Code zu validieren
        SyntaxCheck(code=generated_code)
        return generated_code # Erfolg! Gib den Code direkt zurück.
    except ValidationError as e:
        # Schritt 2: Bei Fehler wird der Korrektur-Prozess gestartet
        print(f"Gate Check 2 (Syntax): Gescheitert! Fehler: {e}\n")
        print("Schritt 3: Korrektur wird versucht...")
        
        error_feedback = str(e)
        system_prompt_refinement = "Du bist ein Experte im Debuggen. Korrigiere den Syntaxfehler. Antworte NUR mit dem korrigierten Code."
        user_prompt_refinement = f"Der Code '{generated_code}' hat einen Fehler: '{error_feedback}'. Korrigiere ihn."
        
        corrected_code = call_llm(client, system_prompt_refinement, user_prompt_refinement, temperature=0.1)

        # SICHERHEITSPRÜFUNG 2: Prüfe auch das Ergebnis des Korrektur-Aufrufs.
        if corrected_code.startswith("API_ERROR:"):
            print("Prozess gestoppt, da der Korrektur-API-Aufruf fehlgeschlagen ist.")
            return None

        try:
            SyntaxCheck(code=corrected_code)
            return corrected_code
        except ValidationError as e_retry:
            print(f"Korrekturversuch ebenfalls gescheitert! Fehler: {e_retry}")
            return None 





# Execution Flow 

## Schritt 1 Plan erstellen inklusive Gate Check mit Pydantic 

In [126]:
# User Promopts and System Prompts
system_prompt_outline = "Du bist ein Programmierassistent. Deine Aufgabe ist es, einen schrittweisen Plan zu erstellen. Antworte NUR mit einer nummerierten Liste."
user_prompt_outline = "Erstelle einen 3-stufigen Umriss für ein Python-Skript, das 'input.csv' liest, den Durchschnitt der Spalte 'Umsatz' berechnet und das Ergebnis in 'output.csv' schreibt."

print(" Starte Prozess: Live Prompt Chaining mit Gate Checks\n" + "="*50)
print("\nSchritt 1: Plan wird generiert...")
print(user_prompt_outline)

# 1. Erstelle den Plan 
outline_text = call_llm(system_prompt_outline, user_prompt_outline)
print("=="*50)
print(f"Generierter Plan:\n{outline_text}")
print("=="*50)


    

 Starte Prozess: Live Prompt Chaining mit Gate Checks

Schritt 1: Plan wird generiert...
Erstelle einen 3-stufigen Umriss für ein Python-Skript, das 'input.csv' liest, den Durchschnitt der Spalte 'Umsatz' berechnet und das Ergebnis in 'output.csv' schreibt.
Generierter Plan:
1. Datei 'input.csv' öffnen und die Daten einlesen, dabei die Spalte 'Umsatz' extrahieren.  
2. Den Durchschnittswert der 'Umsatz'-Spalte berechnen.  
3. Das Ergebnis (Durchschnitt) in die Datei 'output.csv' schreiben.


# Gate Check Pydantic 

In [127]:
# 2. Du Validieren den Plan mit einem Gate Check
print("\nSchritt 2: Validierung des Plans...")
print("=="*50)
is_plan_valid = validate_outline(outline_text)

# 3. Du kannst auf das Ergebnis der Prüfung reagieren
if is_plan_valid:
    print(f"\n**Generierter und validierter Plan:**\n{outline_text}\n")
else:
    print("\nProzess wird wegen fehlerhaftem Plan abgebrochen.")

print("=="*50)


Schritt 2: Validierung des Plans...
Gate Check 1 (Plan-Struktur): Erfolgreich!

**Generierter und validierter Plan:**
1. Datei 'input.csv' öffnen und die Daten einlesen, dabei die Spalte 'Umsatz' extrahieren.  
2. Den Durchschnittswert der 'Umsatz'-Spalte berechnen.  
3. Das Ergebnis (Durchschnitt) in die Datei 'output.csv' schreiben.



## Schritt 2 Auf Basis des validierten Plans Code erstellen inklusive AST Code Check

In [128]:
# 1. Definiere die Prompts für die Code-Generierung hier in der Zelle
system_prompt_code = "Du bist ein Python-Experte. Schreibe Code basierend auf dem gegebenen Umriss. Antworte NUR mit reinem Python-Code."
user_prompt_code = f"Schreibe den Python-Code für den folgenden Umriss:\n{outline_text}"
    
print("\nSchritt 2: Code wird generiert...")
    
# 2. Führe den ersten LLM-Aufruf hier in der Zelle aus
    
generated_code = call_llm(system_prompt=system_prompt_code, user_prompt=user_prompt_code)
print("\n--- Generierter Code ---")
print("=="*50)
print(generated_code)
print("=="*50)    



Schritt 2: Code wird generiert...

--- Generierter Code ---
import csv

umsatzwerte = []

with open('input.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            umsatz = float(row['Umsatz'])
            umsatzwerte.append(umsatz)
        except (ValueError, KeyError):
            continue

if umsatzwerte:
    durchschnitt = sum(umsatzwerte) / len(umsatzwerte)
else:
    durchschnitt = 0

with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Durchschnitt_Umsatz'])
    writer.writerow([durchschnitt])


### AST Code Check 
Wenn der Code fehlschlägt wird er optimiert

In [129]:
# 3. Übergebe den generierten Code an die spezialisierte Validierungs- und Korrektur-Funktion
print("\n--- Validierung und Korrektur werden gestartet ---")
print("=="*50)
final_code = validate_and_refine_code(client, generated_code)

if final_code:
    print("=="*50)
    print(f"\n**Generierter und validierter Code:**\n{final_code}\n")
else:
    print("\nProzess wird wegen fehlerhaftem Code abgebrochen.")

print("=="*50)


--- Validierung und Korrektur werden gestartet ---
Gate Check 2 (Code-Syntax via AST): Erfolgreich!

**Generierter und validierter Code:**
import csv

umsatzwerte = []

with open('input.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            umsatz = float(row['Umsatz'])
            umsatzwerte.append(umsatz)
        except (ValueError, KeyError):
            continue

if umsatzwerte:
    durchschnitt = sum(umsatzwerte) / len(umsatzwerte)
else:
    durchschnitt = 0

with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Durchschnitt_Umsatz'])
    writer.writerow([durchschnitt])



# Finaler Test
Lets execute the created script!

In [131]:

print("Führe den generierten Code aus...")
exec(final_code)
print("✅ Code erfolgreich ausgeführt. 'output.csv' wurde erstellt/aktualisiert.")

Führe den generierten Code aus...
✅ Code erfolgreich ausgeführt. 'output.csv' wurde erstellt/aktualisiert.


# Execution Flow with MAIN

In [ ]:

def main():
    """Orchestrates the chain of live prompts and gate checks."""
    print("🚀 Starte Prozess: Live Prompt Chaining mit Gate Checks\n" + "="*50)

    # --- Step 1: Generate Outline ---
    print("\nSchritt 1: Umriss wird generiert...")
    system_prompt_outline = "Du bist ein Programmierassistent. Deine Aufgabe ist es, einen schrittweisen Plan zu erstellen. Antworte NUR mit einer nummerierten Liste."
    user_prompt_outline = "Erstelle einen 3-stufigen Umriss für ein Python-Skript, das 'input.csv' liest, den Durchschnitt der Spalte 'Umsatz' berechnet und das Ergebnis in 'output.csv' schreibt."
    
    outline_text = call_llm(system_prompt_outline, user_prompt_outline)
    
    try:
        outline_list = [line.strip() for line in outline_text.strip().split('\n') if line.strip()]
        validated_outline = WorkflowPlan(schritte=outline_list)
    except ValidationError as e:
        print(f"❌ Gate Check 1 (Umriss-Struktur): Gescheitert! Fehler: {e}")
        return

    # --- Step 2: Generate Code ---
    print("\nSchritt 2: Code wird generiert...")
    system_prompt_code = "Du bist ein Python-Experte. Schreibe Code basierend auf dem gegebenen Umriss. Verwende die Standardbibliotheken 'csv' und 'statistics'. Antworte NUR mit dem reinen Python-Code und sonst nichts."
    user_prompt_code = f"Schreibe den Python-Code für den folgenden Umriss:\n{outline_text}"
    
    generated_code = call_llm(system_prompt_code, user_prompt_code)
    
    final_code = ""
    try:
        # Erster Versuch der Syntax-Validierung
        SyntaxCheck(code=generated_code)
        final_code = generated_code
    except ValidationError as e:
        print(f"❌ Gate Check 2 (Syntax): Gescheitert! Fehler: {e}\n")
        
        # --- Step 3: Refine Code (Sicherheitsnetz) ---
        print("Schritt 3: Korrektur wird versucht...")
        error_feedback = str(e)
        system_prompt_refinement = "Du bist ein Experte im Debuggen von Python-Code. Korrigiere den Syntaxfehler im folgenden Code. Antworte NUR mit dem korrigierten, reinen Code."
        user_prompt_refinement = f"Der folgende Python-Code enthält einen Syntaxfehler:\n\nCODE:\n{generated_code}\n\nFEHLERMELDUNG:\n{error_feedback}\n\nBitte korrigiere den Code."
        
        corrected_code = call_llm(system_prompt_refinement, user_prompt_refinement, temperature=0.1)

        try:
            SyntaxCheck(code=corrected_code)
            final_code = corrected_code
        except ValidationError as e_retry:
            print(f"❌ Korrekturversuch ebenfalls gescheitert! Fehler: {e_retry}")
            return
            
    if final_code:
        print("\n" + "="*50)
        print("🏁 Prozess erfolgreich abgeschlossen! Finaler, validierter Code:")
        print("-" * 50)
        print(final_code)
        print("-" * 50)
        
        # Führe den validierten Code aus, um die 'output.csv' zu erstellen
        try:
            print("\n⚙️ Führe den generierten Code aus...")
            exec(final_code)
            print("✅ Code erfolgreich ausgeführt. 'output.csv' wurde erstellt/aktualisiert.")
        except Exception as e:
            print(f"🔥 Fehler bei der Ausführung des generierten Codes: {e}")

if __name__ == "__main__":
    main()